# Week 1 - Lab 1: Generate a business idea with Amazon Nova

Small project to showcase using Amazon Nova text generation models.

### Credentials
You will need to set up your AWS credentials in your $HOME/.aws folder or in the .env file. Amazon Bedrock can work with either the standard AWS credentials, or with a Bedrock API key, stored in an environment variable ```AWS_BEARER_TOKEN_BEDROCK```. The API key can be generated from inside Amazon Bedrock console, but it only provides access to Amazon Bedrock. So if you want to use additional AWS Services, you will need to set up your full AWS credentials for CLI and API access in your .env file:
```bash
AWS_ACCESS_KEY_ID=your_access_key
AWS_SECRET_ACCESS_KEY=your_secret_key
```


In [ ]:
# Install necessary packages
# This will also update your pyproject.toml and uv.lock files.
!uv add boto3

In [ ]:
import boto3
import os
from dotenv import load_dotenv
from time import sleep


In [ ]:
# Load api key from .env or environment variable. This notebook is using the simpler API key method, which gives access only to Amazon Bedrock services, instead of standard AWS credentials

load_dotenv(override=True)

os.environ['AWS_BEARER_TOKEN_BEDROCK'] = os.getenv('AWS_BEARER_TOKEN_BEDROCK', 'your-key-if-not-using-env')

region = 'us-east-1' # change to your preferred region - be aware that not all regions have access to all models. If in doubt, use us-east-1.

bedrock = boto3.client(service_name="bedrock", region_name=region) # use this for information and management calls (such as model listings)
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=region) # this is for inference.


In [ ]:
# Let's do a quick test to see if works.
# We will list the available models.

response = bedrock.list_foundation_models()
models = response['modelSummaries']
print(f'AWS Region: {region} - Models:')
for model in models:
    print(f"Model ID: {model['modelId']}, Name: {model['modelName']}")

### Amazon Bedrock Cross-Region Inference
We will use Amazon Nova models for this example.  
  
For inference, we will be using the cross-region inference feature of Amazon Bedrock, which routes the inference call to the region which can best serve it at a given time.  
Cross-region inference [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html)  
For the latest model names using cross-region inference, refer to [Supported Regions and models](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-support.html)  

**Important: Before using a model you need to be granted access to it from the AWS Management Console.**

In [ ]:
# Define the model and message
# Amazon Nova Pro is a multimodal input model - it can be prompted with images and text. We'll only be using text here.

QUESTION = ["I want you to help me pick a business area or industry that might be worth exploring for an Agentic AI opportunity.",
            "Expand on a pain point in that industry that is challenging and ready for an agentic AI solution.",
            "Based on that idea, describe a possible solution"]

BEDROCK_MODEL_ID = 'us.amazon.nova-pro-v1:0'  # try "us.amazon.nova-lite-v1:0" for faster responses.
messages=[]

system_prompt = "You are a helpful business consultant bot. Your responses are succint and professional. You respond in maximum of 4 sentences"

# Function to run a multi-turn conversation. User prompts are stored in the list and we iterate over them, keeping the conversation history to maintain context.

def run_conversation(questions, model_id, system_prompt, sleep_time=5):
    """
    Run a multi-turn conversation with Bedrock model
    Args:
        questions (list): List of questions to ask
        model_id (str): Bedrock model ID to use
        system_prompt (str): System prompt to set context
        sleep_time (int): Time to sleep between requests
    Returns:
        The conversation as a list of dictionaries
    """
    messages = []
    system = [{"text": system_prompt}]

    try:
        for i in range(len(questions)):
            try:
                messages.append({"role": "user", "content": [{"text": questions[i]}]})

                # Make the API call
                response = bedrock_runtime.converse(
                    modelId=model_id,
                    messages=messages, 
                    system=system
                )

                # Store the response
                answer = response['output']['message']['content'][0]['text']

                # Store it into message history
                assistant_message = {"role": "assistant", "content":[{"text":answer}]}
                messages.append(assistant_message)
                print(f"{i}-Question: "+questions[i]+"\nAnswer: " + answer)
                sleep(sleep_time)

            except Exception as e:
                print(f"Error processing question {i}: {str(e)}")
                continue

        return messages

    except Exception as e:
        print(f"Fatal error in conversation: {str(e)}")
        return None


In [27]:
run_conversation(QUESTION,BEDROCK_MODEL_ID,system_prompt=system_prompt)

0-Question: I want you to help me pick a business area or industry that might be worth exploring for an Agentic AI opportunity.
Answer: Consider the healthcare industry for Agentic AI opportunities, focusing on patient care optimization and administrative automation.
1-Question: Expand on a pain point in that industry that is challenging and ready for an agentic AI solution.
Answer: Addressing the challenge of efficient patient scheduling and resource allocation through Agentic AI solutions.
2-Question: Based on that idea, describe a possible solution
Answer: Develop an Agentic AI system to dynamically schedule appointments, optimize staff allocation, and predict patient inflows for healthcare facilities.


[{'role': 'user',
  'content': [{'text': 'I want you to help me pick a business area or industry that might be worth exploring for an Agentic AI opportunity.'}]},
 {'role': 'assistant',
  'content': [{'text': 'Consider the healthcare industry for Agentic AI opportunities, focusing on patient care optimization and administrative automation.'}]},
 {'role': 'user',
  'content': [{'text': 'Expand on a pain point in that industry that is challenging and ready for an agentic AI solution.'}]},
 {'role': 'assistant',
  'content': [{'text': 'Addressing the challenge of efficient patient scheduling and resource allocation through Agentic AI solutions.'}]},
 {'role': 'user',
  'content': [{'text': 'Based on that idea, describe a possible solution'}]},
 {'role': 'assistant',
  'content': [{'text': 'Develop an Agentic AI system to dynamically schedule appointments, optimize staff allocation, and predict patient inflows for healthcare facilities.'}]}]